# Libraries

In [3]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt


from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.utils        import *
from pkg.algorithims  import chc

# Objeto evaluacion

In [4]:
ev = Evaluation()

# Funciones necesarias

## Distancia de Hamming

Calcula la cantidad de genes que se diferencian entre 2 cromosomas, es decir, cuántas posiciones tienen alelos distintos.

In [5]:
padre1 = generate_random_solution()
padre2 = generate_random_solution()

padre1, padre2

(array([16,  8, 18, 16, 18, 12,  6, 10, 16, 12, 16, 10, 12, 18, 16, 16],
       dtype=int64),
 array([19, 13, 13, 11, 15, 11,  4,  9, 15, 15, 17, 17, 17, 13, 11, 19],
       dtype=int64))

In [6]:
distancia_hamming(padre1,padre2)

16

## Operador de cruce

Los cruces en CHC se hacen cambiando exactamente la mitad de los elementos que son distintos en ambos padres y luego se aplica Parent Centered.

### Cruce Parent Centered

1. Selección de los genes a mutar
2. Mutación gaussiana del valor intercambiado.

Mutación gaussiana: Dado un cromosoma p con un gen seleccionado para la mutación i, se le aplica una distribución normal de media el valor a mutar y desviación estandar 2, para que el 68% de las mutaciones estén entre media-2 y media+2

In [7]:
cruce_parent_centered(padre1, padre2)

(array([16, 13, 18, 16, 18, 12,  6,  6, 17, 17, 13, 16, 12, 16, 11, 16],
       dtype=int64),
 array([20, 13, 17, 18, 19, 17,  4,  9, 15, 15, 17, 17, 12, 13, 11, 15],
       dtype=int64))

# Comprobación correcto funcionamiento algoritmo

In [ ]:
np.random.seed(0)

ev.total_calls = 0

cromosoma, fitness = chc(26, 9, 3, ev, True)

In [ ]:
ev.fitness(cromosoma,9)

# Experimentación

In [8]:
# Parametros obtenidos en ag basico
alpha = 9
tama_poblacion = 24

# semillas utilizadas en todos los algoritmos
seeds = np.array([7054, 1354, 23503, 11268, 58283])

# otros parametros
num_rearranques = 5


Obtenemos los datos con los que elaboraremos las tablas comparativas y extraeremos conclusiones.

In [9]:
costs       = np.array([])
evaluations = np.array([])
slots       = np.array([])

for s in seeds:

    # set evauation calls counter to 0
    ev.total_calls = 0

    np.random.seed(s)

    
    crom, fit = chc(
        tama        = tama_poblacion,
        alpha       = alpha,
        rearranques = num_rearranques,
        ev= ev
    )

    print(crom)
    print(fit)

    value, total_distance, slot = ev.fitness(crom,alpha)
    
    costs       = np.append(costs, total_distance)
    evaluations = np.append(evaluations, ev.total_calls)
    slots       = np.append(slots, slot)


df = pd.DataFrame({
    "ev_medias"    : [evaluations.mean()],
    "ev_mejor"     : [evaluations.min()],
    "ev_std"       : [evaluations.std()],
    "coste_medio"  : [costs.mean()],
    "coste_mejor"  : [costs.min()],
    "coste_std"    : [costs.std()],
    "slots_medio"  : [slots.mean()],
    "slots_min"    : [slots.min()],
    "slots_std"    : [slots.std()]
})

df_2 = pd.DataFrame({
    'Ejecución'    : [1,2,3,4,5],
    "Coste mejor"  : costs,
    "Evaluaciones" : evaluations,
    "slots"        : slots
})

[18 12 20 21 20 14 12 10 12 14 14 24  2 14  5 11]
2344.7874161816253
[18 12 17 22 19 14 12 11 12 16 14 24  2 10  7 13]
2347.658947921934
[18 12 21 19 19 14 12  9 12 15 16 22  2  7 13 12]
2349.4557411959595
[18 12 20 21 20 14 11 11 12 13 13 24  2  8 13 11]
2347.4475964774037
[13 12 20 22 18 14 15  7 12 14 19 23  2 16  4 12]
2350.504227987584


In [10]:
df

,ev_medias,ev_mejor,ev_std,coste_medio,coste_mejor,coste_std,slots_medio,slots_min,slots_std
0,13461.4,11257.0,1200.43752,340.970786,337.787416,1.955926,223.0,223.0,0.0


In [11]:
df_2

,Ejecución,Coste mejor,Evaluaciones,slots
0,1,337.787416,14381.0,223.0
1,2,340.658948,14441.0,223.0
2,3,342.455741,13117.0,223.0
3,4,340.447596,14111.0,223.0
4,5,343.504228,11257.0,223.0


In [12]:
df.to_excel('./01_stats_p2/chc_stats.xlsx')
df_2.to_excel('./01_stats_p2/chc_ejecuciones.xlsx')